In [1]:
!unzip /content/artifacts.zip


Archive:  /content/artifacts.zip
  inflating: best_model.h5           
  inflating: scaler.joblib           
  inflating: label_encoder.joblib    
  inflating: activity_labels.txt     


In [3]:
!pip install -q fastapi uvicorn joblib numpy pandas tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model

# Apply nest_asyncio to allow uvicorn to run inside Jupyter/Colab
nest_asyncio.apply()

# Define request and response schemas
class PredictionRequest(BaseModel):
    features: list  # list of floats length num_features

class PredictionResponse(BaseModel):
    predicted_label: str
    probabilities: dict

# Initialize FastAPI app
app = FastAPI(
    title="HAR Prediction API",
    description="API for Human Activity Recognition model predictions",
    version="1.0.0"
)

# Load artifacts on startup using lifespan
@app.on_event("startup")
async def load_artifacts():
    global scaler, label_encoder, model, activity_labels
    scaler = joblib.load("/content/scaler.joblib")
    label_encoder = joblib.load("/content/label_encoder.joblib")
    model = load_model("/content/best_model.h5")
    activity_labels = pd.read_csv(
        "/content/activity_labels.txt",
        sep=" ", header=None, names=["code", "label"]
    )

# Prediction endpoint
@app.post("/predict", response_model=PredictionResponse)
async def predict(req: PredictionRequest):
    num_features = model.input_shape[1]
    if len(req.features) != num_features:
        raise HTTPException(
            status_code=400,
            detail=f"Expected {num_features} features, got {len(req.features)}"
        )
    X = np.array(req.features).reshape(1, -1)
    X_scaled = scaler.transform(X)
    probs = model.predict(X_scaled)[0]
    class_idx = int(np.argmax(probs))
    code = class_idx + 1
    label = activity_labels.loc[activity_labels.code == code, 'label'].values[0]
    prob_dict = {activity_labels.loc[i, 'label']: float(probs[i]) for i in range(len(probs))}
    return PredictionResponse(predicted_label=label, probabilities=prob_dict)

# Health check endpoint
@app.get("/health")
async def health_check():
    return {"status": "ok"}

# Launch the server in Colab
if __name__ == "__main__":
    # Note: change port if needed
    uvicorn.run(app, host="0.0.0.0", port=8000)


<ipython-input-5-dc401c794da9>:30: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
INFO:     Started server process [171]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [ ]:
#––– 1) Install requests (if not already) –––
!pip install -q requests

#––– 2) Imports and loading raw test data –––
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import StandardScaler
import joblib

# Paths (adjust if necessary)
DATASET_PATH = "/content/UCI HAR Dataset"
SCALER_PATH  = "/content/scaler.joblib"

# Load raw X_test
X_test = pd.read_csv(
    f"{DATASET_PATH}/test/X_test.txt",
    delim_whitespace=True,
    header=None
).values

# Load your saved scaler
scaler = joblib.load(SCALER_PATH)

# Scale test set
X_test_scaled = scaler.transform(X_test)

#––– 3) Prepare the sample and call the API –––
API_URL = "http://127.0.0.1:8000/predict"  # or your ngrok URL

# Use the first test sample
sample_features = list(X_test_scaled[0])  # this is now defined

payload = {
    "features": sample_features
}

resp = requests.post(API_URL, json=payload)

if resp.status_code == 200:
    result = resp.json()
    print("Predicted Activity:", result["predicted_label"])
    print("\nClass Probabilities:")
    for label, prob in result["probabilities"].items():
        print(f"  {label}: {prob:.4f}")
else:
    print(f"Error {resp.status_code}: {resp.text}")
